In [3]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

import os
import pandas as pd
from torchvision.io import read_image

In [4]:
torch.cuda.is_available()

False

In [5]:
device = torch.device('cuda:0')

In [6]:
def encode_label(label):
    labels = ['Snail', 'Water', 'Plant', 'Cattle', 'Grain', 'Bird', 'People', 'Animal', 'Snake']
    for i in range(len(labels)):
        if label == labels[i]:
            return i
    
    return -1

def decode_label(code):
    labels = ['Snail', 'Water', 'Plant', 'Cattle', 'Grain', 'Bird', 'People', 'Animal', 'Snake']
    return labels[code]
    

In [ ]:
# Self-defined dataset class
class HieroglyphicsDataset(Dataset):
    def __init__(self, directory_path, mode, transform=None, target_transform=None):
        assert(mode == 'train' or mode == 'test')
        
        self.data = []
        self.transform = transform
        self.target_transform = target_transform
        
        print('You are creating a %s dataset from %s.' % (mode, directory_path))
              
        for _, categories, _ in os.walk(directory_path):
            for category in categories:
                label = encode_label(category)
                label = torch.tensor(label)
                if self.target_transform:
                    label = self.target_transform(label)
                    
                category_path = os.path.join(directory_path, category)
                for _, _, image_files in os.walk(category_path):
              
                    # Split dataset into training and testing subsets
                    divider = (int)(len(image_files)*0.75)
                    if mode == 'train':
                        image_files = image_files[:divider]
                    elif mode == 'test':
                        image_files = image_files[divider:]
                    
                    # Add image 
                    for image_file in image_files:
                        image_path = os.path.join(category_path, image_file)
                        image = read_image(image_path)/255
                        if self.transform:
                            image = self.transform(image)
                        self.data.append((image, label))

                    # Summarize
                    print('\t%s - %s' % (category, str(len(image_files))))
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
trainset = HieroglyphicsDataset('./dataset', 'train', transform=transforms.Compose([
    transforms.Resize((224, 224))]),
)
testset = HieroglyphicsDataset('./dataset', 'test', transform=transforms.Compose([
    transforms.Resize((224, 224))]),)

# Load Data

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(trainset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(testset, batch_size=4, shuffle=True)

In [ ]:
# Check
train_features, train_labels = next(iter(train_dataloader))
img = train_features[0][1] # We only visualize one channel since all the channels are the same
label = train_labels[0]

plt.imshow(img)
plt.show()
print(f"Label: {decode_label(label)}")

print(img)

# Train

In [ ]:
# Related modules
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
# Hyper parameters
lr = 0.01
momentum = 0.9
epochs = 40

In [ ]:
class MyNet(nn.Module):
    def __init__(self, category_num):
        super(MyNet, self).__init__()
        self.resnet = torchvision.models.resnet50(pretrained=True)
        self.end = nn.Linear(1000, category_num, bias=True)
        
    def forward(self, x):
        x = self.resnet(x)
        x = self.end(x)
        
        return x

In [ ]:
gpus = [0]
model = MyNet(category_num=9)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [ ]:
def train(model, criterion, optimizer, epoch, train_loader):
    model.train()
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()
        

        # forward + backward + optimize
        outputs = model(inputs)
        labels = labels.cuda()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

In [ ]:
for epoch in range(epochs):
    train(model, criterion, optimizer, epoch, train_dataloader)

In [ ]:
# Save model
PATH = './models/egyptian_net.pth'
torch.save(model.state_dict(), PATH)

# Test

In [1]:
def test(model, test_loader):
    model.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = model(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the %d test images: %d %%' % (total, 
        100 * correct / total))

def test_category(model, test_loader):
    model.eval()
    
    classes = ['Snail', 'Water', 'Plant', 'Cattle', 'Grain', 'Bird', 'People', 'Animal', 'Snake']
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}
    
    # again no gradients needed
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)
            # collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1


    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print("Accuracy for class {:5s} is: {:.1f} % ({} / {})".format(
                classname,
                accuracy,
                correct_count,
                total_pred[classname])
             )

In [2]:
# Load model
PATH = './models/egyptian_net.pth'
model = MyNet(category_num=9)
model.load_state_dict(torch.load(PATH))

NameError: name 'MyNet' is not defined

In [ ]:
test(model, test_dataloader)

In [ ]:
test_category(model, test_dataloader)